# 엔드투엔드 NLP: 뉴스 헤드라인 분류기(로컬 버전)

_**네 가지 도메인 간의 뉴스 헤드라인을 분류하기 위해 Keras 기반 모델 학습**_

이 노트북은 SageMaker 스튜디오의 `Python 3 (TensorFlow 2.3 Python 3.7 CPU Optimized)` 커널 또는 클래식 SageMaker 노트북 인스턴스의 `conda_tensorflow2_p37`에서 잘 작동합니다.


---

이 버전에서는 노트북 인스턴스 자체에서 모델을 학습하고 평가합니다. 다음 노트북에서는 이러한 인프라 요구 사항을 분리하기 위해 Amazon SageMaker를 활용하는 방법을 보여드리겠습니다.

pip 버전에 대한 경고는 무시해도 됩니다.



In [ ]:
# First install some libraries which might not be available across all kernels (e.g. in Studio):
!pip install "ipywidgets<8"

### News Aggregator 데이터셋 다운로드

[AWS의 오픈 데이터 레지스트리](https://registry.opendata.aws/fast-ai-nlp/) 퍼블릭 리포지토리에서 **FastAI AG 뉴스** 데이터 세트를 다운로드합니다. 이 데이터 세트에는 뉴스 헤드라인과 그에 해당하는 클래스의 표가 포함되어 있습니다.


In [ ]:
%%time
local_dir = "data"
# Download the AG News data from the Registry of Open Data on AWS.
!mkdir -p {local_dir}
!aws s3 cp s3://fast-ai-nlp/ag_news_csv.tgz {local_dir} --no-sign-request

# Un-tar the AG News data.
!tar zxf {local_dir}/ag_news_csv.tgz -C {local_dir}/ --strip-components=1 --no-same-owner
print("Done!")

### 데이터 세트를 시각화해 봅시다.

데이터 처리 작업을 위해 ag_news_csv/train.csv 파일을 Pandas 데이터 프레임에 로드하겠습니다.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import re

import numpy as np
import pandas as pd
import util.preprocessing

In [ ]:
column_names = ["CATEGORY", "TITLE", "CONTENT"]
# we use the train.csv only
df = pd.read_csv(f"{local_dir}/train.csv", names=column_names, header=None, delimiter=",")
# shuffle the DataFrame rows
df = df.sample(frac=1, random_state=1337)
# make the category classes more readable
mapping = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tech"}
df = df.replace({"CATEGORY": mapping})
df.head()

이 연습에서는 **아래 내용만 사용하겠습니다**:

- 뉴스 기사의 **제목**(헤드라인)을 입력으로 사용합니다.
- 대상 변수로 **카테고리**를 사용합니다.



In [ ]:
df["CATEGORY"].value_counts()

데이터 세트에는 동일한 가중치를 가진 **4개의 기사 카테고리**가 있습니다:

- Business
- Sci/Tech
- Sports
- World


## 자연어 전처리

텍스트 데이터를 알고리즘이 모델을 생성하는 데 사용할 수 있는 숫자 형식으로 변환하기 위해 몇 가지 기본 처리를 수행합니다.

라벨 더미 인코딩, 문서 토큰화, 입력 특징 차원에 대한 고정 시퀀스 길이 설정, 고정 길이 입력 벡터를 갖도록 문서 패딩과 같은 NLP 워크로드에 대한 일반적인 사전 처리를 수행합니다.



### 더미 인코딩 레이블 만들기

In [ ]:
encoded_y, labels = util.preprocessing.dummy_encode_labels(df, "CATEGORY")
print(labels)
print(encoded_y)

예를 들어, (셔플된) 데이터 프레임의 첫 번째 레코드를 살펴봅니다:

In [ ]:
df["CATEGORY"].iloc[0]

In [ ]:
encoded_y[0]

###  토큰화 및 고정 시퀀스 길이 설정하기

개별 문자가 아닌 보다 의미 있는 단어 수준에서 입력(개별 문자가 아닌)을 설명하고, 입력 특징 차원의 고정된 길이를 보장하고자 합니다.

In [ ]:
processed_docs, tokenizer = util.preprocessing.tokenize_and_pad_docs(df, "TITLE")

In [ ]:
df["TITLE"].iloc[0]

In [ ]:
processed_docs[0]

### 단어 임베딩 가져오기

단어를 숫자 형태로 표현하기 위해 어휘의 각 단어에 대해 미리 학습된 벡터 표현을 사용합니다: 이 경우 영어 이외의 다양한 언어에도 사용할 수 있는 [FastText의 사전 학습된 단어 임베딩](https://fasttext.cc/docs/en/crawl-vectors.html)을 사용할 것입니다.

SageMaker에 내장된 [BlazingText algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html)을 사용하여 도메인별 맞춤 단어 임베딩을 학습할 수도 있습니다. 방법을 보여주는 예제 노트북은 공식 [blazingtext_word2vec_text8 sample](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/blazingtext_word2vec_text8)을 참조하세요.

In [ ]:
%%time
embedding_matrix = util.preprocessing.get_word_embeddings(tokenizer, f"{local_dir}/embeddings")

In [ ]:
np.save(
    file=f"{local_dir}/embeddings/docs-embedding-matrix",
    arr=embedding_matrix,
    allow_pickle=False,
)
vocab_size = embedding_matrix.shape[0]
print(embedding_matrix.shape)

### 학습 및 테스트 세트 분할

마지막으로 데이터를 모델 훈련 세트와 평가 세트로 나눠야 합니다:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    processed_docs,
    encoded_y,
    test_size=0.2,
    random_state=42,
)

In [ ]:
# Do you always remember to save your datasets for traceability when experimenting locally? ;-)
os.makedirs(f"{local_dir}/train", exist_ok=True)
np.save(f"{local_dir}/train/train_X.npy", X_train)
np.save(f"{local_dir}/train/train_Y.npy", y_train)
os.makedirs(f"{local_dir}/test", exist_ok=True)
np.save(f"{local_dir}/test/test_X.npy", X_test)
np.save(f"{local_dir}/test/test_Y.npy", y_test)

## 모델 정의하기

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Embedding, Flatten, MaxPooling1D
from tensorflow.keras.models import Sequential

seed = 42
np.random.seed(seed)
num_classes = len(labels)

In [ ]:
model = Sequential()
model.add(
    Embedding(
        embedding_matrix.shape[0],  # Final vocabulary size
        embedding_matrix.shape[1],  # Word vector dimensions
        weights=[embedding_matrix],
        input_length=40,
        trainable=False,
        name="embed",
    )
)
model.add(Conv1D(filters=128, kernel_size=3, activation="relu", name="conv_1"))
model.add(MaxPooling1D(pool_size=5, name="maxpool_1"))
model.add(Flatten(name="flat_1"))
model.add(Dropout(0.3, name="dropout_1"))
model.add(Dense(128, activation="relu", name="dense_1"))
model.add(Dense(num_classes, activation="softmax", name="out_1"))

# Compile the model
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["acc"])

model.summary()

## 모델 맞춤(훈련) 및 평가하기

In [ ]:
%%time
# fit the model here in the notebook:
print("Training model")
model.fit(X_train, y_train, batch_size=16, epochs=5, verbose=1)
print("Evaluating model")
# TODO: Better differentiate train vs val loss in logs
scores = model.evaluate(X_test, y_test, verbose=2)
print(
    "Validation results: "
    + "; ".join(
        map(lambda i: f"{model.metrics_names[i]}={scores[i]:.5f}", range(len(model.metrics_names)))
    )
)

## 모델 사용(로컬)

몇 가지 예제 헤드라인으로 모델을 평가해 봅시다...

위젯을 사용하는 데 어려움이 있다면 파이썬에서 `classify()` 함수를 호출하면 됩니다. 헤드라인을 창의적으로 만들 수 있습니다!


In [ ]:
import ipywidgets as widgets
from IPython import display
from tensorflow.keras.preprocessing.sequence import pad_sequences


def classify(text):
    """Classify a headline and print the results"""
    encoded_example = tokenizer.texts_to_sequences([text])
    # Pad documents to a max length of 40 words
    max_length = 40
    padded_example = pad_sequences(encoded_example, maxlen=max_length, padding="post")
    result = model.predict(padded_example)
    print(result)
    ix = np.argmax(result)
    print(f"Predicted class: '{labels[ix]}' with confidence {result[0][ix]:.2%}")


interaction = widgets.interact_manual(
    classify,
    text=widgets.Text(
        value="The markets were bullish after news of the merger",
        placeholder="Type a news headline...",
        description="Headline:",
        layout=widgets.Layout(width="99%"),
    ),
)
interaction.widget.children[1].description = "Classify!"

In [ ]:
# Or just use the function to classify your own headline:
classify("Retailers are expanding after the recent economic growth")

## 검토

이 노트북에서는 공개적으로 다운로드할 수 있는 데이터를 사전 처리하고 신경망 뉴스 헤드라인 분류기 모델을 학습했습니다: 데이터 과학자가 로컬 컴퓨터에서 작업할 때 일반적으로 하는 것처럼 말입니다.

...하지만 클라우드를 더 효과적으로 사용해 고성능 리소스를 할당하고, 학습된 모델을 다른 애플리케이션에서 사용할 수 있도록 쉽게 배포할 수 있을까요?

다음 노트북인 [Headline Classifier SageMaker.ipynb](Headline%20Classifier%20SageMaker.ipynb)로 이동하여 동일한 모델을 학습한 다음 Amazon SageMaker를 사용하여 특정 대상 인프라에 배포하는 방법을 보여드리겠습니다.
